# Data Cleaning and Preparation

In [44]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno  # For missing value visualization
from scipy import stats
from scipy.stats import ttest_ind, chi2_contingency
from sklearn.feature_selection import mutual_info_classif

import warnings
warnings.filterwarnings("ignore")

# sets the maximum number of columns that will be displayed in a dataframe to 200 for improved data viewing
pd.set_option('display.max_columns', None)
# Set the display option to show all columns
pd.set_option('display.max_info_columns', 200)

In [45]:
# Custom color palette inspired by CUSTOM
CUSTOM_PALETTE = ["#4E79A7", "#F28E2B", "#E15759", "#76B7B2", "#59A14F", "#EDC948", "#B07AA1", "#FF9DA7", "#9C755F", "#BAB0AC"]
sns.set_palette(CUSTOM_PALETTE)
plt.style.use('ggplot')

# Seaborn style
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1.15, rc={"grid.linewidth": 0.6})

# Matplotlib rcParams
plt.style.use('default')
plt.rcParams.update({
    'font.family': 'DejaVu Sans',  # Or try 'Segoe UI', 'Calibri', etc.
    'font.weight': 'normal',
    'axes.titlesize': 16,
    'axes.titleweight': 'bold',
    'axes.titlepad': 25,
    'axes.labelsize': 14,
    'axes.labelweight': 'regular',
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'grid.color': '#D3D3D3',
    'grid.alpha': 0.5,
    'grid.linestyle': '--',
    'figure.figsize': (10, 6),
    'figure.dpi': 100,
    'axes.prop_cycle': plt.cycler(color=CUSTOM_PALETTE)
})

# Despine
sns.despine(offset=10, trim=True)

<Figure size 1000x600 with 0 Axes>

### Data Reading

Let's start by loading the dataset and reviewing its structure. Note the data types (e.g., numerical, categorical, datetime) and identify the key variables relevant to the analysis. This helps me set expectations for the cleaning and preprocessing tasks.

In [46]:
df = pd.read_csv('../data/raw/Telecom_Data.csv')

### Data Exploration
Data exploration is a critical step in the data analysis process, where you examine the dataset to gain a preliminary understanding of the data, detect patterns, and identify potential issues that may need further investigation. Data exploration is important because it helps to provide a solid foundation for subsequent data analysis tasks, hypothesis testing and data visualization.

In [47]:
# Data Reading
df.head()

,Customer ID,Month,Month of Joining,zip_code,Gender,Age,Married,Dependents,Number of Dependents,Location ID,Service ID,state,county,timezone,area_codes,country,latitude,longitude,arpu,roam_ic,roam_og,loc_og_t2t,loc_og_t2m,loc_og_t2f,loc_og_t2c,std_og_t2t,std_og_t2m,std_og_t2f,std_og_t2c,isd_og,spl_og,og_others,loc_ic_t2t,loc_ic_t2m,loc_ic_t2f,std_ic_t2t,std_ic_t2m,std_ic_t2f,std_ic_t2o,spl_ic,isd_ic,ic_others,total_rech_amt,total_rech_data,vol_4g,vol_5g,arpu_5g,arpu_4g,night_pck_user,fb_user,aug_vbc_5g,Churn Value,Referred a Friend,Number of Referrals,Phone Service,Multiple Lines,Internet Service,Internet Type,Streaming Data Consumption,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Payment Method,Status ID,Satisfaction Score,Churn Category,Churn Reason,Customer Status,offer
0,hthjctifkiudi0,1,1.0,71638,Female,36.000000,No,No,0.0,jeavwsrtakgq0,bfbrnsqreveeuafgps0,AR,Chicot County,America/Chicago,870.0,US,33.52,-91.43,273.07,18.88,78.59,280.32,30.970000,5.71,1.79,25.71,175.56,0.470000,0.0,5.11,0.65,13.99,121.51,168.40,67.61,115.69,52.22,18.71,0.0,0.26,11.53,46.42,18.0,NaN,38.30,219.25,Not Applicable,Not Applicable,NaN,NaN,214.99,1,Yes,9.0,Yes,Yes,Yes,DSL,27,No,No,Yes,Yes,No,Yes,Yes,Yes,Credit Card,vvhwtmkbxtvsppd52013,3,Competitor,Competitor offered higher download speeds,Churned,A
1,uqdtniwvxqzeu1,6,6.0,72566,Male,36.657198,No,No,0.0,qcvetdmalnkw1,tkqnsqflrdatnqapsh1,AR,Izard County,America/Chicago,870.0,US,36.22,-92.08,-329.96,69.46,72.08,255.73,148.800000,30.00,7.61,308.29,265.20,10.820000,0.0,1.23,905.51,1.69,212.93,155.19,29.04,9.15,38.89,0.84,0.0,0.05,32.51,25.53,1183.0,0.0,0.00,0.00,0.0,0.0,0.0,1.0,0.00,0,No,0.0,Yes,Yes,No,NaN,14,No,Yes,No,No,Yes,No,No,No,Bank Withdrawal,jucxaluihiluj82863,4,Not Applicable,Not Applicable,Stayed,F
2,uqdtniwvxqzeu1,7,6.0,72566,Male,36.607828,No,No,0.0,qcvetdmalnkw1,tkqnsqflrdatnqapsh1,AR,Izard County,America/Chicago,870.0,US,36.22,-92.08,101.22,1012.60,115.26,52.95,1151.693063,103.28,15.71,244.20,15.19,61.837211,0.0,13.14,455.15,115.63,121.80,699.39,44.49,83.59,914.70,13.25,0.0,0.06,13.05,5.62,295.0,7.0,14.83,967.95,-9.4,106.3,1.0,1.0,85.87,0,Yes,6.0,Yes,No,Yes,Cable,82,No,No,Yes,No,Yes,No,No,Yes,Credit Card,vjskkxphumfai57182,3,Not Applicable,Not Applicable,Stayed,No Offer
3,uqdtniwvxqzeu1,8,6.0,72566,Male,36.943638,No,No,0.0,qcvetdmalnkw1,tkqnsqflrdatnqapsh1,AR,Izard County,America/Chicago,870.0,US,36.22,-92.08,215.48,84.18,99.85,140.51,4006.990000,280.86,6.33,346.14,103.15,183.530000,0.0,33.88,495.60,14.01,658.96,195.02,144.11,50.18,2.35,623.94,0.0,0.07,69.13,10.62,354.0,1.0,264.90,268.11,-5.15,77.53,0.0,1.0,268.38,0,Yes,10.0,Yes,No,Yes,Fiber Optic,57,No,No,Yes,No,Yes,No,No,Yes,Wallet Balance,cdwbcrvylqca53109,4,Not Applicable,Not Applicable,Stayed,J
4,uqdtniwvxqzeu1,9,6.0,72566,Male,36.632494,No,No,0.0,qcvetdmalnkw1,tkqnsqflrdatnqapsh1,AR,Izard County,America/Chicago,870.0,US,36.22,-92.08,636.55,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.0,0.00,0.00,0.00,138.85,201.92,19.89,15.91,23.78,16.01,0.0,0.03,64.35,36.18,0.0,NaN,52.78,370.59,Not Applicable,Not Applicable,NaN,NaN,399.84,0,Yes,1.0,No,No,Yes,Fiber Optic,38,No,No,No,No,No,Yes,No,Yes,Credit Card,whqrmeulitfj98550,1,Not Applicable,Not Applicable,Stayed,No Offer


In [48]:
# Overall dataset shape
print(f"\nDataset Shape: {df.shape[0]} rows, {df.shape[1]} columns")


Dataset Shape: 653753 rows, 74 columns


I will start converting all the dataset columsn into a lower case since they are mixed and replace the space with understcode '_' 

In [49]:
# Convert column names to lowercase and replace spaces with underscores
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.head()

,customer_id,month,month_of_joining,zip_code,gender,age,married,dependents,number_of_dependents,location_id,service_id,state,county,timezone,area_codes,country,latitude,longitude,arpu,roam_ic,roam_og,loc_og_t2t,loc_og_t2m,loc_og_t2f,loc_og_t2c,std_og_t2t,std_og_t2m,std_og_t2f,std_og_t2c,isd_og,spl_og,og_others,loc_ic_t2t,loc_ic_t2m,loc_ic_t2f,std_ic_t2t,std_ic_t2m,std_ic_t2f,std_ic_t2o,spl_ic,isd_ic,ic_others,total_rech_amt,total_rech_data,vol_4g,vol_5g,arpu_5g,arpu_4g,night_pck_user,fb_user,aug_vbc_5g,churn_value,referred_a_friend,number_of_referrals,phone_service,multiple_lines,internet_service,internet_type,streaming_data_consumption,online_security,online_backup,device_protection_plan,premium_tech_support,streaming_tv,streaming_movies,streaming_music,unlimited_data,payment_method,status_id,satisfaction_score,churn_category,churn_reason,customer_status,offer
0,hthjctifkiudi0,1,1.0,71638,Female,36.000000,No,No,0.0,jeavwsrtakgq0,bfbrnsqreveeuafgps0,AR,Chicot County,America/Chicago,870.0,US,33.52,-91.43,273.07,18.88,78.59,280.32,30.970000,5.71,1.79,25.71,175.56,0.470000,0.0,5.11,0.65,13.99,121.51,168.40,67.61,115.69,52.22,18.71,0.0,0.26,11.53,46.42,18.0,NaN,38.30,219.25,Not Applicable,Not Applicable,NaN,NaN,214.99,1,Yes,9.0,Yes,Yes,Yes,DSL,27,No,No,Yes,Yes,No,Yes,Yes,Yes,Credit Card,vvhwtmkbxtvsppd52013,3,Competitor,Competitor offered higher download speeds,Churned,A
1,uqdtniwvxqzeu1,6,6.0,72566,Male,36.657198,No,No,0.0,qcvetdmalnkw1,tkqnsqflrdatnqapsh1,AR,Izard County,America/Chicago,870.0,US,36.22,-92.08,-329.96,69.46,72.08,255.73,148.800000,30.00,7.61,308.29,265.20,10.820000,0.0,1.23,905.51,1.69,212.93,155.19,29.04,9.15,38.89,0.84,0.0,0.05,32.51,25.53,1183.0,0.0,0.00,0.00,0.0,0.0,0.0,1.0,0.00,0,No,0.0,Yes,Yes,No,NaN,14,No,Yes,No,No,Yes,No,No,No,Bank Withdrawal,jucxaluihiluj82863,4,Not Applicable,Not Applicable,Stayed,F
2,uqdtniwvxqzeu1,7,6.0,72566,Male,36.607828,No,No,0.0,qcvetdmalnkw1,tkqnsqflrdatnqapsh1,AR,Izard County,America/Chicago,870.0,US,36.22,-92.08,101.22,1012.60,115.26,52.95,1151.693063,103.28,15.71,244.20,15.19,61.837211,0.0,13.14,455.15,115.63,121.80,699.39,44.49,83.59,914.70,13.25,0.0,0.06,13.05,5.62,295.0,7.0,14.83,967.95,-9.4,106.3,1.0,1.0,85.87,0,Yes,6.0,Yes,No,Yes,Cable,82,No,No,Yes,No,Yes,No,No,Yes,Credit Card,vjskkxphumfai57182,3,Not Applicable,Not Applicable,Stayed,No Offer
3,uqdtniwvxqzeu1,8,6.0,72566,Male,36.943638,No,No,0.0,qcvetdmalnkw1,tkqnsqflrdatnqapsh1,AR,Izard County,America/Chicago,870.0,US,36.22,-92.08,215.48,84.18,99.85,140.51,4006.990000,280.86,6.33,346.14,103.15,183.530000,0.0,33.88,495.60,14.01,658.96,195.02,144.11,50.18,2.35,623.94,0.0,0.07,69.13,10.62,354.0,1.0,264.90,268.11,-5.15,77.53,0.0,1.0,268.38,0,Yes,10.0,Yes,No,Yes,Fiber Optic,57,No,No,Yes,No,Yes,No,No,Yes,Wallet Balance,cdwbcrvylqca53109,4,Not Applicable,Not Applicable,Stayed,J
4,uqdtniwvxqzeu1,9,6.0,72566,Male,36.632494,No,No,0.0,qcvetdmalnkw1,tkqnsqflrdatnqapsh1,AR,Izard County,America/Chicago,870.0,US,36.22,-92.08,636.55,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.0,0.00,0.00,0.00,138.85,201.92,19.89,15.91,23.78,16.01,0.0,0.03,64.35,36.18,0.0,NaN,52.78,370.59,Not Applicable,Not Applicable,NaN,NaN,399.84,0,Yes,1.0,No,No,Yes,Fiber Optic,38,No,No,No,No,No,Yes,No,Yes,Credit Card,whqrmeulitfj98550,1,Not Applicable,Not Applicable,Stayed,No Offer


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 653753 entries, 0 to 653752
Data columns (total 74 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   customer_id                 653753 non-null  object 
 1   month                       653753 non-null  int64  
 2   month_of_joining            653753 non-null  float64
 3   zip_code                    653753 non-null  int64  
 4   gender                      653753 non-null  object 
 5   age                         653753 non-null  float64
 6   married                     653753 non-null  object 
 7   dependents                  653753 non-null  object 
 8   number_of_dependents        648808 non-null  float64
 9   location_id                 653753 non-null  object 
 10  service_id                  653753 non-null  object 
 11  state                       653753 non-null  object 
 12  county                      653753 non-null  object 
 13  timezone      

- We have some columns with missing values.
- Other columns are in the wrong data type and need to be converted right data types 

In [53]:
# Basic statistics for numeric variables
numeric_df = df.select_dtypes(include=['int64', 'float64'])
numeric_stats = numeric_df.describe()
numeric_stats

,month,month_of_joining,zip_code,age,number_of_dependents,latitude,longitude,arpu,roam_ic,roam_og,loc_og_t2t,loc_og_t2m,loc_og_t2f,loc_og_t2c,std_og_t2t,std_og_t2m,std_og_t2f,std_og_t2c,isd_og,spl_og,og_others,loc_ic_t2t,loc_ic_t2m,loc_ic_t2f,std_ic_t2t,std_ic_t2m,std_ic_t2f,std_ic_t2o,spl_ic,isd_ic,ic_others,total_rech_amt,total_rech_data,vol_4g,vol_5g,night_pck_user,fb_user,aug_vbc_5g,churn_value,number_of_referrals,streaming_data_consumption,satisfaction_score
count,653753.000000,653753.000000,653753.000000,653753.000000,648808.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.0,653753.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.00000,653753.000000,653753.000000,653753.000000,653753.0,653753.000000,653753.000000,653753.000000,653753.000000,443849.000000,653753.000000,653753.000000,280650.000000,243359.000000,653753.000000,653753.000000,653366.000000,653753.000000,653753.000000
mean,9.508209,5.823574,90386.180128,36.627508,1.161307,37.652731,-114.392353,781.196280,249.649581,267.554917,834.627412,678.554324,32.269385,30.328614,577.606835,441.072131,34.878469,0.0,49.607795,88.964864,98.301011,846.742888,720.748656,330.91955,369.434031,306.380190,125.712838,0.0,0.251306,251.281481,144.004310,1687.052445,4.852747,192.096023,2240.746081,0.217638,0.795796,530.530351,0.045682,4.337965,27.574284,3.131712
std,3.298742,2.855148,8412.506653,12.168678,2.254298,4.687151,10.537540,1807.269437,424.616652,625.610182,1589.594440,1112.490267,59.017015,67.610974,1306.895141,990.212679,57.920915,0.0,113.245301,168.700830,161.822471,1216.216840,1009.687483,483.25191,631.201763,437.613681,185.826256,0.0,0.438407,442.322591,293.132647,2980.015423,8.319981,592.101187,4582.943857,0.412640,0.403120,1402.826222,0.208796,3.769501,26.349552,1.249076
min,1.000000,1.000000,71601.000000,19.000000,0.000000,31.790000,-124.630000,-2258.680000,-25.049540,-108.651017,-30.166077,0.000000,-6.936413,0.000000,0.000000,0.000000,-2.204748,0.0,0.000000,-42.624578,-12.866150,0.000000,0.000000,0.00000,-40.955947,0.000000,0.000000,0.0,0.000000,-49.514184,-58.342240,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-371.472900,0.000000,0.000000,0.000000,1.000000
25%,7.000000,3.000000,88424.000000,28.000000,0.000000,34.140000,-121.650000,118.940000,12.090000,14.710000,32.700000,26.260000,1.460000,1.610000,33.120000,25.560000,1.200000,0.0,3.250000,4.940000,3.430000,85.570000,84.170000,36.11000,42.460000,32.190000,12.460000,0.0,0.040000,26.980000,20.330000,72.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000,3.000000
50%,10.000000,6.000000,93032.000000,34.000000,0.000000,36.080000,-118.390000,348.540000,50.560000,75.100000,171.330000,135.460000,7.800000,8.180000,174.600000,134.800000,6.340000,0.0,17.190000,25.580000,17.830000,171.490000,168.390000,72.06000,84.470000,64.760000,24.980000,0.0,0.080000,53.700000,40.540000,374.000000,0.000000,47.010000,274.110000,0.000000,1.000000,117.320000,0.000000,4.000000,20.000000,3.000000
75%,12.000000,8.000000,95551.000000,43.000000,1.000000,38.600000,-108.800000,580.650000,162.030000,135.290000,309.090000,618.230000,14.090000,14.700000,316.240000,244.490000,36.640000,0.0,31.140000,46.190000,106.780000,1259.260000,1090.080000,496.79000,126.270000,448.820000,186.720000,0.0,0.210000,80.370000,60.730000,1089.000000,7.000000,154.900000,895.810000,0.000000,1.000000,311.720000,0.000000,8.000000,49.000000,4.000000
max,14.000000,12.000000,99403.000000,80.000000,9.000000,48.990000,-89.740000,9394.500000,1719.430000,3161.780000,6431.250000,4212.010000,283.530000,336.130000,7366.160000,5622.540000,217.440000,0.0,765.050000,1020.710000,609.800000,4363.950000,3846.547156,1872.34000,2527.070000,1619.680000,663.930000,0.0,2.330000,1917.461595,1344.130000,11900.000000,32.000000,4503.930000,19876.750000,1.000000,1.000000,8214.870000,1.000000,11.000000,85.0000

In [54]:
# Basic statistics for categorical variables
cat_df = df.select_dtypes(exclude=['int64', 'float64'])
cat_stats = numeric_df.describe()
cat_stats

,month,month_of_joining,zip_code,age,number_of_dependents,latitude,longitude,arpu,roam_ic,roam_og,loc_og_t2t,loc_og_t2m,loc_og_t2f,loc_og_t2c,std_og_t2t,std_og_t2m,std_og_t2f,std_og_t2c,isd_og,spl_og,og_others,loc_ic_t2t,loc_ic_t2m,loc_ic_t2f,std_ic_t2t,std_ic_t2m,std_ic_t2f,std_ic_t2o,spl_ic,isd_ic,ic_others,total_rech_amt,total_rech_data,vol_4g,vol_5g,night_pck_user,fb_user,aug_vbc_5g,churn_value,number_of_referrals,streaming_data_consumption,satisfaction_score
count,653753.000000,653753.000000,653753.000000,653753.000000,648808.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.0,653753.000000,653753.000000,653753.000000,653753.000000,653753.000000,653753.00000,653753.000000,653753.000000,653753.000000,653753.0,653753.000000,653753.000000,653753.000000,653753.000000,443849.000000,653753.000000,653753.000000,280650.000000,243359.000000,653753.000000,653753.000000,653366.000000,653753.000000,653753.000000
mean,9.508209,5.823574,90386.180128,36.627508,1.161307,37.652731,-114.392353,781.196280,249.649581,267.554917,834.627412,678.554324,32.269385,30.328614,577.606835,441.072131,34.878469,0.0,49.607795,88.964864,98.301011,846.742888,720.748656,330.91955,369.434031,306.380190,125.712838,0.0,0.251306,251.281481,144.004310,1687.052445,4.852747,192.096023,2240.746081,0.217638,0.795796,530.530351,0.045682,4.337965,27.574284,3.131712
std,3.298742,2.855148,8412.506653,12.168678,2.254298,4.687151,10.537540,1807.269437,424.616652,625.610182,1589.594440,1112.490267,59.017015,67.610974,1306.895141,990.212679,57.920915,0.0,113.245301,168.700830,161.822471,1216.216840,1009.687483,483.25191,631.201763,437.613681,185.826256,0.0,0.438407,442.322591,293.132647,2980.015423,8.319981,592.101187,4582.943857,0.412640,0.403120,1402.826222,0.208796,3.769501,26.349552,1.249076
min,1.000000,1.000000,71601.000000,19.000000,0.000000,31.790000,-124.630000,-2258.680000,-25.049540,-108.651017,-30.166077,0.000000,-6.936413,0.000000,0.000000,0.000000,-2.204748,0.0,0.000000,-42.624578,-12.866150,0.000000,0.000000,0.00000,-40.955947,0.000000,0.000000,0.0,0.000000,-49.514184,-58.342240,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-371.472900,0.000000,0.000000,0.000000,1.000000
25%,7.000000,3.000000,88424.000000,28.000000,0.000000,34.140000,-121.650000,118.940000,12.090000,14.710000,32.700000,26.260000,1.460000,1.610000,33.120000,25.560000,1.200000,0.0,3.250000,4.940000,3.430000,85.570000,84.170000,36.11000,42.460000,32.190000,12.460000,0.0,0.040000,26.980000,20.330000,72.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000,3.000000
50%,10.000000,6.000000,93032.000000,34.000000,0.000000,36.080000,-118.390000,348.540000,50.560000,75.100000,171.330000,135.460000,7.800000,8.180000,174.600000,134.800000,6.340000,0.0,17.190000,25.580000,17.830000,171.490000,168.390000,72.06000,84.470000,64.760000,24.980000,0.0,0.080000,53.700000,40.540000,374.000000,0.000000,47.010000,274.110000,0.000000,1.000000,117.320000,0.000000,4.000000,20.000000,3.000000
75%,12.000000,8.000000,95551.000000,43.000000,1.000000,38.600000,-108.800000,580.650000,162.030000,135.290000,309.090000,618.230000,14.090000,14.700000,316.240000,244.490000,36.640000,0.0,31.140000,46.190000,106.780000,1259.260000,1090.080000,496.79000,126.270000,448.820000,186.720000,0.0,0.210000,80.370000,60.730000,1089.000000,7.000000,154.900000,895.810000,0.000000,1.000000,311.720000,0.000000,8.000000,49.000000,4.000000
max,14.000000,12.000000,99403.000000,80.000000,9.000000,48.990000,-89.740000,9394.500000,1719.430000,3161.780000,6431.250000,4212.010000,283.530000,336.130000,7366.160000,5622.540000,217.440000,0.0,765.050000,1020.710000,609.800000,4363.950000,3846.547156,1872.34000,2527.070000,1619.680000,663.930000,0.0,2.330000,1917.461595,1344.130000,11900.000000,32.000000,4503.930000,19876.750000,1.000000,1.000000,8214.870000,1.000000,11.000000,85.0000

### Let's check the columns with high missing values

In [55]:
# Calculate missing values count
missing_count = df.isna().sum()
    
# Calculate missing values percentage (normalized)
missing_percent = df.isna().sum()/len(df) * 100
    
missing_summary = pd.DataFrame({
        'Missing Values': missing_count,
        'Missing Percentage': missing_percent.round(2)
    })
missing_summary = missing_summary.sort_values(by='Missing Percentage', ascending=False)
missing_summary.head(8)

,Missing Values,Missing Percentage
fb_user,410394,62.78
night_pck_user,373103,57.07
internet_type,325250,49.75
total_rech_data,209904,32.11
multiple_lines,46080,7.05
unlimited_data,11103,1.70
number_of_dependents,4945,0.76
number_of_referrals,387,0.06


- Some of these columns like `fb_user`, `night_pck_user`, are not that useful to the analysis and prediction.
- We deep more these columns we care to findout the mising values

In [56]:
# Null values in total recharge data
int(df['total_rech_data'].isna().sum())

209904

In [57]:
# Null values in Internet Type
int(df['internet_type'].isna().sum())

325250

In [58]:
# Missing value percentage
float(df['total_rech_data'].isna().sum()/df.shape[0])

0.321075390858627

**Observation:**
- The missing values may indicate customers who haven't recharged their accounts or whose recharge data wasn't recorded. 
- It's possible that customers with missing data received free data services and didn't need to recharge.
- Alternatively, the missing values could be the result of data recording errors or system issues.

In [59]:
# Checking the value counts of Internet Service where total recharge data was null
df[df['total_rech_data'].isna()]['internet_service'].value_counts(dropna=False)

internet_service
Yes    209904
Name: count, dtype: int64

**Observation:** 

- All customers with missing recharge data have opted for internet service. 
- The next step should be to check whether these customers have actually used the internet service or not. 
- This could help determine if the missing recharge data is due to free internet service usage or other factors.

In [60]:
# Let's check unlimited data column
df[(df['total_rech_data'].isna())]['unlimited_data'].value_counts().reset_index()

,unlimited_data,count
0,Yes,181040
1,No,28864


In [61]:
# Lets check Average Revenue for 4g and 5g
df[(df['total_rech_data'].isna())][['arpu_4g','arpu_5g']].value_counts().reset_index()

,arpu_4g,arpu_5g,count
0,Not Applicable,Not Applicable,195182
1,297.57,8530.983628675234,4
2,1850.89,8566.14608743311,3
3,1885.23,8566.92344099435,3
4,1811.33,8565.25056888673,3
...,...,...,...
14242,-10.33,1566.64,1
14243,-1.17,2027.3,1
14244,-0.23,1485.61,1
14245,-7.14,2120.92,1


**Observation:**
- We can fill the missing values in the **total_rech_data** column with 0 when the **arpu_4g** or **arpu_5g** is "Not Applicable." 
- This is because **arpu** is a measure of the revenue generated per user, and if it is not applicable, it may indicate that the user is not generating any revenue for the company. 
- In such cases, it is reasonable to assume that the total recharge data is 0.

In [62]:
# Check the value counts of ARPU 4g and 5g
df[['arpu_4g','arpu_5g']].value_counts().reset_index()

,arpu_4g,arpu_5g,count
0,Not Applicable,Not Applicable,195182
1,0.0,0.0,184117
2,0.0,63.0,13024
3,63.0,0.0,12969
4,254687.0,0.0,10911
...,...,...,...
195840,-0.04,128.23,1
195841,-0.04,20.97,1
195842,-0.04,88.04,1
195843,-0.05,-6.86,1


In [63]:
# Replacing all values of total recharge data= 0 where arpu 4g and 5g are not applicable
df.loc[(df['arpu_4g']=='Not Applicable') | (df['arpu_5g']=='Not Applicable'),'total_rech_data']=0

In [64]:
# Missing value percentage
float(df['total_rech_data'].isna().sum()/df.shape[0])

0.022519208324856637

**Observation**
- The missing value percentage in the **total_rech_data** column is approximately 2.25%. 
- We cannot fill the other missing values with 0 because they have some **ARPU** to consider, indicating that these customers are generating revenue and potentially using the service.

In [65]:
# Calculate the mean of 'total_rech_data' where either 'arpu_4g' or 'arpu_5g' is not equal to 'Not Applicable'
float(df.loc[(df['arpu_4g']!='Not Applicable') | (df['arpu_5g']!='Not Applicable'),'total_rech_data'].mean())

4.85274721808543

**Observation**
- The mean of **total_rech_data** for customers where either **arpu_4g** or **arpu_5g** is not marked as "Not Applicable" is approximately 4.85. 
- We will use this mean value to fill the NaN values in the **total_rech_data** column.

In [66]:
# Fill NaN values in 'total_rech_data' with the mean of 'total_rech_data' where either 'arpu_4g' or 'arpu_5g' is not equal to 'Not Applicable'
df['total_rech_data']=df['total_rech_data'].fillna(df.loc[(df['arpu_4g']!='Not Applicable') | (df['arpu_5g']!='Not Applicable'),'total_rech_data'].mean())


In [67]:
# Check the value counts for Internet Type
df['internet_type'].value_counts(dropna=False).reset_index()

,internet_type,count
0,NaN,325250
1,Fiber Optic,134991
2,Cable,112100
3,DSL,81412


In [68]:
# Check value counts for Internet Service where Internet Type is null
df[df['internet_type'].isna()]['internet_service'].value_counts(dropna=False).reset_index()

,internet_service,count
0,No,236152
1,Yes,89098


**Observation**
- All null values in the **Internet Type** column correspond to customers who do not have **Internet Service**.
- Therefore, we will fill these null values with "Not Applicable" to accurately reflect the absence of internet service for these customers.

In [70]:
# Filling Null values in Internet Type 
df['internet_type']=df['internet_type'].fillna('Not Applicable')

In [74]:
# Insert a new column named 'total_recharge' before the last column in the dataframe 
# The values of 'total_recharge' are the sum of 'total_rech_amt' and 'total_rech_data'
df.insert(loc=df.shape[1]-1,column='total_recharge',value=df['total_rech_amt']+df['total_rech_data'])

- Replace 'Not Applicable' with 0 in both 'arpu_4g' and 'arpu_5g'.

In [75]:
# Replace 'Not Applicable' with 0 in 'arpu_4g'
df['arpu_4g'] = df['arpu_4g'].replace('Not Applicable', 0)
# Replace 'Not Applicable' with 0 in 'arpu_5g'
df['arpu_5g'] = df['arpu_5g'].replace('Not Applicable', 0)
# Convert 'arpu_4g' to float data type
df['arpu_4g'] = df['arpu_4g'].astype(float)
# Convert 'arpu_5g' to float data type
df['arpu_5g'] = df['arpu_5g'].astype(float)

In [76]:
df.head()

,customer_id,month,month_of_joining,zip_code,gender,age,married,dependents,number_of_dependents,location_id,service_id,state,county,timezone,area_codes,country,latitude,longitude,arpu,roam_ic,roam_og,loc_og_t2t,loc_og_t2m,loc_og_t2f,loc_og_t2c,std_og_t2t,std_og_t2m,std_og_t2f,std_og_t2c,isd_og,spl_og,og_others,loc_ic_t2t,loc_ic_t2m,loc_ic_t2f,std_ic_t2t,std_ic_t2m,std_ic_t2f,std_ic_t2o,spl_ic,isd_ic,ic_others,total_rech_amt,total_rech_data,vol_4g,vol_5g,arpu_5g,arpu_4g,night_pck_user,fb_user,aug_vbc_5g,churn_value,referred_a_friend,number_of_referrals,phone_service,multiple_lines,internet_service,internet_type,streaming_data_consumption,online_security,online_backup,device_protection_plan,premium_tech_support,streaming_tv,streaming_movies,streaming_music,unlimited_data,payment_method,status_id,satisfaction_score,churn_category,churn_reason,customer_status,total_recharge,offer
0,hthjctifkiudi0,1,1.0,71638,Female,36.000000,No,No,0.0,jeavwsrtakgq0,bfbrnsqreveeuafgps0,AR,Chicot County,America/Chicago,870.0,US,33.52,-91.43,273.07,18.88,78.59,280.32,30.970000,5.71,1.79,25.71,175.56,0.470000,0.0,5.11,0.65,13.99,121.51,168.40,67.61,115.69,52.22,18.71,0.0,0.26,11.53,46.42,18.0,0.0,38.30,219.25,0.00,0.00,NaN,NaN,214.99,1,Yes,9.0,Yes,Yes,Yes,DSL,27,No,No,Yes,Yes,No,Yes,Yes,Yes,Credit Card,vvhwtmkbxtvsppd52013,3,Competitor,Competitor offered higher download speeds,Churned,18.0,A
1,uqdtniwvxqzeu1,6,6.0,72566,Male,36.657198,No,No,0.0,qcvetdmalnkw1,tkqnsqflrdatnqapsh1,AR,Izard County,America/Chicago,870.0,US,36.22,-92.08,-329.96,69.46,72.08,255.73,148.800000,30.00,7.61,308.29,265.20,10.820000,0.0,1.23,905.51,1.69,212.93,155.19,29.04,9.15,38.89,0.84,0.0,0.05,32.51,25.53,1183.0,0.0,0.00,0.00,0.00,0.00,0.0,1.0,0.00,0,No,0.0,Yes,Yes,No,Not Applicable,14,No,Yes,No,No,Yes,No,No,No,Bank Withdrawal,jucxaluihiluj82863,4,Not Applicable,Not Applicable,Stayed,1183.0,F
2,uqdtniwvxqzeu1,7,6.0,72566,Male,36.607828,No,No,0.0,qcvetdmalnkw1,tkqnsqflrdatnqapsh1,AR,Izard County,America/Chicago,870.0,US,36.22,-92.08,101.22,1012.60,115.26,52.95,1151.693063,103.28,15.71,244.20,15.19,61.837211,0.0,13.14,455.15,115.63,121.80,699.39,44.49,83.59,914.70,13.25,0.0,0.06,13.05,5.62,295.0,7.0,14.83,967.95,-9.40,106.30,1.0,1.0,85.87,0,Yes,6.0,Yes,No,Yes,Cable,82,No,No,Yes,No,Yes,No,No,Yes,Credit Card,vjskkxphumfai57182,3,Not Applicable,Not Applicable,Stayed,302.0,No Offer
3,uqdtniwvxqzeu1,8,6.0,72566,Male,36.943638,No,No,0.0,qcvetdmalnkw1,tkqnsqflrdatnqapsh1,AR,Izard County,America/Chicago,870.0,US,36.22,-92.08,215.48,84.18,99.85,140.51,4006.990000,280.86,6.33,346.14,103.15,183.530000,0.0,33.88,495.60,14.01,658.96,195.02,144.11,50.18,2.35,623.94,0.0,0.07,69.13,10.62,354.0,1.0,264.90,268.11,-5.15,77.53,0.0,1.0,268.38,0,Yes,10.0,Yes,No,Yes,Fiber Optic,57,No,No,Yes,No,Yes,No,No,Yes,Wallet Balance,cdwbcrvylqca53109,4,Not Applicable,Not Applicable,Stayed,355.0,J
4,uqdtniwvxqzeu1,9,6.0,72566,Male,36.632494,No,No,0.0,qcvetdmalnkw1,tkqnsqflrdatnqapsh1,AR,Izard County,America/Chicago,870.0,US,36.22,-92.08,636.55,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.0,0.00,0.00,0.00,138.85,201.92,19.89,15.91,23.78,16.01,0.0,0.03,64.35,36.18,0.0,0.0,52.78,370.59,0.00,0.00,NaN,NaN,399.84,0,Yes,1.0,No,No,Yes,Fiber Optic,38,No,No,No,No,No,Yes,No,Yes,Credit Card,whqrmeulitfj98550,1,Not Applicable,Not Applicable,Stayed,0.0,No Offer


Outlier detection is a crucial step in data analysis that helps identify and handle data points significantly different from the rest. Addressing outliers ensures more accurate and reliable results in statistical analyses and machine learning models.

**Understanding Outliers:**

Outliers are data points that deviate markedly from the majority of a dataset. They can result from variability in the data or errors during data collection. Identifying these outliers is essential, as they can skew statistical analyses and affect the performance of machine learning models.

**Methods for Outlier Detection:**

There are two primary approaches for detecting outliers:

1. **Parametric Methods:**

   Parametric methods assume that the data follows a specific distribution, such as a normal distribution. A common technique is the Z-score method, which measures how many standard deviations a data point is from the mean. Data points with a Z-score greater than 3 or less than -3 are typically considered outliers.

   *Formula:*

   $$ Z = \frac{(X - \mu)}{\sigma} $$

   Where:
   - $Z$ is the Z-score.
   - $X$ is the data point.
   - $mu$ is the mean of the dataset.
   - $sigma$ is the standard deviation of the dataset.

2. **Non-Parametric Methods:**

   Non-parametric methods do not assume any specific data distribution. One widely used technique is based on the Interquartile Range (IQR). The IQR measures the range between the first quartile (25th percentile) and the third quartile (75th percentile) of the data. Data points outside the range defined by:

   $$ Q1 - 1.5 \times IQR $$

   and

   $$ Q3 + 1.5 \times IQR $$

   are considered outliers.

   Where:
   - $Q1$ is the first quartile. $0.25$
   - $Q3$ is the third quartile. $0.75$
   - $IQR$ is the Interquartile Range $Q3$ - $Q1$

**Visualizing Outliers:**

Boxplots are effective tools for visualizing outliers. They display the distribution of data based on five summary statistics: minimum, first quartile, median, third quartile, and maximum. Outliers appear as individual points outside the "whiskers" of the boxplot.

![Boxplot Example](../references/1WhSq1aAoIa3XILagZV2scg.png)

*Figure: Boxplot illustrating the distribution of data and highlighting outliers.*

**Further Reading:**

For a more in-depth understanding of outlier detection techniques, consider exploring the following resources:

- [Writing mathematical expressions in Markdown](https://docs.github.com/en/get-started/writing-on-github/working-with-advanced-formatting/writing-mathematical-expressions)
- [Math and equations in Jupyter Book](https://jupyterbook.org/content/math.html)
- [Markdown Cheat Sheet](https://www.markdownguide.org/cheat-sheet/)

By employing appropriate outlier detection methods and understanding their underlying principles, you can enhance the quality and reliability of your data analyses.

## Outlier Detection

**What Are Outliers?**

Outliers are unusual data points that stand far apart from most observations in your dataset. They represent either rare but legitimate events or potential errors in your data. Addressing outliers properly is crucial for accurate analysis and reliable predictions.

## Impact of Outliers

* **Distort averages** - Pull statistics away from true central tendencies
* **Create false patterns** - Generate misleading relationships between variables
* **Bias models** - Cause algorithms to overemphasize exceptional cases
* **Disrupt grouping** - Interfere with customer segmentation efforts
* **Compromise forecasts** - Lead to unrealistic predictions

**Detection Methods**

**Statistical approaches** measure how far points deviate from expected patterns:

* **Z-score method** - Identifies points based on standard deviations from the mean
* **IQR method** - Uses quartiles to establish boundaries for normal values

The right detection technique depends on your data's natural distribution and your specific business context.

Effective outlier management ensures your analysis captures genuine patterns rather than being derailed by exceptional cases or errors.

**Read more about Outliers**

- [What are outliers in the data?](https://www.itl.nist.gov/div898/handbook/prc/section1/prc16.htm#:~:text=An%20outlier%20is%20an%20observation,what%20will%20be%20considered%20abnormal.)

- [How to Find Outliers in Statistics](https://www.freecodecamp.org/news/what-is-an-outlier-definition-and-how-to-find-outliers-in-statistics/)
- [What Is an Outlier?](https://careerfoundry.com/en/blog/data-analytics/what-is-an-outlier/)

**Using Quantiles for Outlier Detection: A Strategic Choice**

Quantiles offer a robust framework for identifying outliers without distribution assumptions. By dividing data into equal portions, they establish natural boundaries based on the data's actual spread. The first quartile (Q1) marks where 25% of values fall below, while the third quartile (Q3) indicates where 75% fall below.

![](../references/1WhSq1aAoIa3XILagZV2scg.png)

In [81]:
# List of continuous columns
cont_cols=['age','number_of_dependents',
       'roam_ic', 'roam_og', 'loc_og_t2t',
       'loc_og_t2m', 'loc_og_t2f', 'loc_og_t2c', 'std_og_t2t', 'std_og_t2m',
       'std_og_t2f', 'std_og_t2c', 'isd_og', 'spl_og', 'og_others',
       'loc_ic_t2t', 'loc_ic_t2m', 'loc_ic_t2f', 'std_ic_t2t', 'std_ic_t2m',
       'std_ic_t2f', 'std_ic_t2o', 'spl_ic', 'isd_ic', 'ic_others',
       'total_rech_amt', 'total_rech_data', 'vol_4g', 'vol_5g', 'arpu_5g',
       'arpu_4g', 'arpu', 'aug_vbc_5g', 'number_of_referrals','satisfaction_score',
       'streaming_data_consumption']  

In [82]:
# Create an empty dataframe with columns as cont_cols and index as quantiles
quantile_df=pd.DataFrame(columns=cont_cols,index=[0.1,0.25,0.5,0.75,0.8,0.9,0.95,0.97,0.99])

# for each column in cont_cols, calculate the corresponding quantiles and store them in the quantile_df
for col in cont_cols:
   quantile_df[col]=df[col].quantile([0.1,0.25,0.5,0.75,0.8,0.9,0.95,0.97,0.99])

In [84]:
# Let's check out the quantiles df
quantile_df

,age,number_of_dependents,roam_ic,roam_og,loc_og_t2t,loc_og_t2m,loc_og_t2f,loc_og_t2c,std_og_t2t,std_og_t2m,std_og_t2f,std_og_t2c,isd_og,spl_og,og_others,loc_ic_t2t,loc_ic_t2m,loc_ic_t2f,std_ic_t2t,std_ic_t2m,std_ic_t2f,std_ic_t2o,spl_ic,isd_ic,ic_others,total_rech_amt,total_rech_data,vol_4g,vol_5g,arpu_5g,arpu_4g,arpu,aug_vbc_5g,number_of_referrals,satisfaction_score,streaming_data_consumption
0.10,24.0,0.0,0.000000,0.000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.0000,34.740000,33.790000,14.4600,16.950000,13.060000,5.030,0.0,0.02,10.770000,8.100000,0.0,0.000000,0.0000,0.0000,0.000000,0.000000,-256.2000,0.0000,0.0,1.0,0.0
0.25,28.0,0.0,12.090000,14.710,32.700000,26.2600,1.460000,1.6100,33.120000,25.560000,1.20,0.0,3.250000,4.940000,3.4300,85.570000,84.170000,36.1100,42.460000,32.190000,12.460,0.0,0.04,26.980000,20.330000,72.0,0.000000,0.0000,0.0000,0.000000,0.000000,118.9400,0.0000,0.0,3.0,2.0
0.50,34.0,0.0,50.560000,75.100,171.330000,135.4600,7.800000,8.1800,174.600000,134.800000,6.34,0.0,17.190000,25.580000,17.8300,171.490000,168.390000,72.0600,84.470000,64.760000,24.980,0.0,0.08,53.700000,40.540000,374.0,0.000000,47.0100,274.1100,0.000000,0.000000,348.5400,117.3200,4.0,3.0,20.0
0.75,43.0,1.0,162.030000,135.290,309.090000,618.2300,14.090000,14.7000,316.240000,244.490000,36.64,0.0,31.140000,46.190000,106.7800,1259.260000,1090.080000,496.7900,126.270000,448.820000,186.720,0.0,0.21,80.370000,60.730000,1089.0,2.000000,154.9000,895.8100,194.470000,228.220000,580.6500,311.7200,8.0,4.0,49.0
0.80,47.0,2.0,496.935485,146.820,856.841431,1392.8500,43.876522,15.9700,344.970000,266.540000,71.61,0.0,33.910000,50.240000,229.2460,1999.702000,1471.760000,653.0600,543.125842,634.003448,275.200,0.0,0.33,384.834562,64.800000,2197.0,4.852747,176.3600,1654.5460,789.000000,783.290000,626.2300,350.5000,8.0,4.0,56.0
0.90,55.0,4.0,969.048000,689.608,3614.015883,2644.5980,126.595429,109.0980,1547.160000,1007.728354,143.14,0.0,113.181983,372.788742,382.7200,2974.572396,2424.836361,1198.6180,1525.968000,1030.546647,466.868,0.0,0.71,1102.783115,532.388000,7013.0,14.000000,219.2680,9658.3760,2219.752000,2224.100000,1901.5140,789.0000,10.0,5.0,69.0
0.95,61.0,7.0,1283.198000,1954.392,5079.830000,3479.4380,183.494000,207.5140,3953.491029,3108.254000,171.80,0.0,319.290206,470.140200,489.7000,3719.724000,3166.706000,1462.3020,2022.064442,1360.444000,569.740,0.0,1.27,1443.949075,914.270266,9369.0,23.000000,663.2040,14517.6400,8530.865147,8675.302558,5892.6180,3943.2100,11.0,5.0,77.0
0.97,64.0,8.0,1494.043200,2550.390,5806.054400,3756.4444,206.744400,277.3444,5344.227818,3848.534961,188.88,0.0,394.208100,518.359307,531.5700,3911.517600,3468.838800,1657.1832,2145.504400,1476.410000,594.000,0.0,1.75,1554.890000,1212.837600,10492.0,26.000000,1438.5100,16580.3764,8724.440600,8839.721689,7592.5688,5949.3792,11.0,5.0,80.0
0.99,74.0,9.0,1646.899600,3041.760,6191.204000,4060.2988,257.650000,311.4648,6729.403200,4875.008539,208.18,0.0,637.009600,836.140000,579.3748,4200.448800,3679.364400,1792.9848,2434.554800,1571.760000,639.000,0.0,2.19,1601.920000,1317.510000,11367.0,30.000000,4289.8496,18614.5528,254687.000000,254687.000000,8846.9584,7366.7684,11.0,5.0,83.0


**Outlier Obervations**

- I noticed some significant outliers in `vol_5g` and `arpu_4g/arpu_5g` columns that could impact our modeling efforts.
- These outliers highlight the need to investigate further to distinguish between legitimate high spenders and potential errors in the data.

In [85]:
# Checking further
df['arpu_4g'].quantile([0.75,0.8,0.9,0.95,0.97,0.99,0.999])

0.750       228.220000
0.800       783.290000
0.900      2224.100000
0.950      8675.302558
0.970      8839.721689
0.990    254687.000000
0.999    254687.000000
Name: arpu_4g, dtype: float64

**Insights**

Looking at the Average Revenue Per User for 4G:

* Most customers (75%) spend less than 228 units
* A small group (10%) spends between 2,224 and 8,839 units
* The top 1% shows an extreme jump to 254,687 units

- We should verify these extreme values before analysis and consider separate treatment for high-value customers.

In [86]:
# Calculate the proportion of rows in the DataFrame where the value in the 'arpu_4g' column is equal to 254687
df[df['arpu_4g']==254687].shape[0]/df.shape[0]

0.019651152652454366

**ARPU 4G Extreme Value**

The data shows **1.97%** of customers have an identical, extremely high ARPU 4G value of **254,687** units. This is significant because:

* This single value appears in nearly 2% of all records
* The identical amount across multiple customers is statistically improbable
* The value is over 1,000× higher than typical customer spending

This pattern strongly suggests a data quality issue rather than legitimate revenue—likely a placeholder, error code, or processing issue. These records should be investigated and potentially treated as missing values to prevent analysis distortion.

In [87]:
# Let's check it out
df[df['arpu_4g']==254687].head()

,customer_id,month,month_of_joining,zip_code,gender,age,married,dependents,number_of_dependents,location_id,service_id,state,county,timezone,area_codes,country,latitude,longitude,arpu,roam_ic,roam_og,loc_og_t2t,loc_og_t2m,loc_og_t2f,loc_og_t2c,std_og_t2t,std_og_t2m,std_og_t2f,std_og_t2c,isd_og,spl_og,og_others,loc_ic_t2t,loc_ic_t2m,loc_ic_t2f,std_ic_t2t,std_ic_t2m,std_ic_t2f,std_ic_t2o,spl_ic,isd_ic,ic_others,total_rech_amt,total_rech_data,vol_4g,vol_5g,arpu_5g,arpu_4g,night_pck_user,fb_user,aug_vbc_5g,churn_value,referred_a_friend,number_of_referrals,phone_service,multiple_lines,internet_service,internet_type,streaming_data_consumption,online_security,online_backup,device_protection_plan,premium_tech_support,streaming_tv,streaming_movies,streaming_music,unlimited_data,payment_method,status_id,satisfaction_score,churn_category,churn_reason,customer_status,total_recharge,offer
9,uqdtniwvxqzeu1,14,6.0,72566,Male,36.463423,No,No,0.0,qcvetdmalnkw1,tkqnsqflrdatnqapsh1,AR,Izard County,America/Chicago,870.0,US,36.22,-92.08,1330.04,1582.05,157.20,161.810000,1827.38,39.79,1.00,1362.59,5267.31,171.81,0.0,390.32,24.940000,511.23,2128.61,2896.11,54.41,100.540000,585.44,162.70,0.0,0.11,10.460000,1247.37,255.0,0.0,0.0,0.0,254687.0,254687.0,0.0,1.0,0.0,0,Yes,9.0,Yes,No,No,Not Applicable,74,No,No,Yes,No,Yes,No,No,No,Credit Card,inebwpymzwpup39698,4,Not Applicable,Not Applicable,Stayed,255.0,No Offer
86,ucpurmfkdlnwi18,13,12.0,71747,Female,20.000000,Yes,No,0.0,rqiqguxisfoc18,dkupusivpzrazcfsdi18,AR,Union County,America/Chicago,870.0,US,33.04,-92.18,160.07,18.63,31.29,2894.413815,834.78,209.17,9.59,177.64,116.17,120.34,0.0,14.74,439.340381,100.81,156.27,254.19,29.68,998.828718,24.13,12.62,0.0,0.29,795.194692,5.06,8462.0,0.0,0.0,0.0,0.0,254687.0,0.0,1.0,0.0,0,Yes,6.0,Yes,Yes,No,Not Applicable,0,No,No,No,Yes,No,No,No,No,Bank Withdrawal,usfobpyxwqrkg27554,5,Not Applicable,Not Applicable,Stayed,8462.0,No Offer
103,sirifvlkipkel21,13,11.0,92865,Female,40.000000,Yes,No,0.0,jobplwgowgko21,zmuwwsnfbwxxdxzuvz21,CA,Orange County,America/Los_Angeles,714.0,US,33.83,-117.85,478.77,26.04,72.49,111.050000,1.87,6.89,4.83,11.50,134.28,6.71,0.0,31.44,6.230000,2.70,171.28,167.16,15.18,54.880000,64.06,31.83,0.0,0.01,41.910000,61.24,417.0,0.0,0.0,0.0,0.0,254687.0,NaN,NaN,0.0,0,Yes,0.0,Yes,Yes,No,Not Applicable,56,No,Yes,Yes,No,Yes,Yes,Yes,No,Credit Card,cullucfodcpbc24549,3,Not Applicable,Not Applicable,Stayed,417.0,No Offer
112,dnnrchjlmrylq24,14,9.0,91423,Female,48.000000,Yes,Yes,0.0,vxainqiqplai24,liroqcvpdnrzdyolqw24,CA,Los Angeles County,America/Los_Angeles,2.13e+17,US,34.14,-118.42,143.68,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.000000,0.00,149.18,2769.19,207.23,33.720000,331.07,3.33,0.0,0.06,0.090000,2.56,0.0,0.0,0.0,0.0,0.0,254687.0,NaN,NaN,0.0,0,Yes,6.0,No,Yes,No,Not Applicable,51,No,Yes,Yes,No,No,Yes,Yes,No,Bank Withdrawal,qflywarsexbpg13676,4,Not Applicable,Not Applicable,Stayed,0.0,G
145,pltaycxycbhvo31,11,7.0,95126,Other,35.000000,No,No,0.0,sjmjgqjvhvth31,xbmtjtsvypinczxnhf31,CA,Santa Clara County,America/Los_Angeles,408.0,US,37.32,-121.91,95.40,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.000000,0.00,3210.57,525.28,136.57,19.790000,1.21,202.92,0.0,0.05,61.380000,52.97,0.0,0.0,0.0,0.0,0.0,254687.0,NaN,1.0,0.0,0,Yes,10.0,No,No,No,Cable,56,No,Yes,No,No,No,Yes,Yes,No,Bank Withdrawal,xayhhjriwxte83055,3,Not Applicable,Not Applicable,Stayed,0.0,J


In [88]:
# Get the value counts of 'total_rech_data' for observations where the value in the 'arpu_4g' column is equal to 254687
df[df['arpu_4g']==254687]['total_rech_data'].value_counts()

total_rech_data
0.0    12847
Name: count, dtype: int64

**High ARPU 4G Records**

All **12,847** records with the extreme ARPU 4G value of **254,687** have exactly **0.0** for their total data recharges.

Meaning these are definitely anomalous records because:

* A customer with extremely high 4G revenue should logically have some data recharges
* The perfect correlation (100% of these records show zero recharges) indicates a systematic issue
* The large number of affected records (12,847) suggests a significant data processing problem

- This pattern strongly confirms these values are not legitimate customer spending but rather a data error or placeholder
- These records should be excluded from revenue analysis or have their ARPU values recoded as missing to prevent severe distortion of analytical results.

In [89]:
# Replace the outlier value 254687 in the 'arpu_4g' column of the dataframe 'df' with 0.
df['arpu_4g']=df['arpu_4g'].replace(254687,0)

In [90]:
# Checking further
df['arpu_4g'].quantile([0.75,0.8,0.9,0.95,0.97,0.99,0.999])

0.750      120.570000
0.800      504.112000
0.900     1893.758000
0.950     2493.880000
0.970     8675.470757
0.990     8839.721689
0.999    87978.000000
Name: arpu_4g, dtype: float64

**ARPU 4G Distribution After Outlier Treatment**

After replacing the extreme outlier value (254,687) with 0, the ARPU 4G distribution shows significant changes:

* The 75th percentile dropped from 228.22 to 120.57 units
* The 90th percentile decreased from 2,224.10 to 1,893.76 units
* The 99th percentile fell dramatically from 254,687 to 8,839.72 units
* The highest extreme value (99.9th percentile) is now 87,978 units

Meaning:

1. The distribution is still skewed but much more realistic
2. The extreme outliers were artificially inflating even the 75th percentile
3. A clearer segmentation of customer spending emerges with distinct tiers
4. The remaining high values (88K at 99.9%) may need further investigation but appear less systematic

This corrected distribution provides a more reliable foundation for customer value analysis, segmentation, and predictive modeling. The dramatic changes in percentile values confirm the earlier outliers were indeed erroneous data rather than legitimate high-spending customers.

In [91]:
# Filter by 'arpu_4g' value of 87978 and count unique values in 'total_rech_data' column
df[df['arpu_4g']==87978]['total_rech_data'].value_counts()

total_rech_data
0.0    5007
Name: count, dtype: int64

All **5,007** records with the ARPU 4G value of **87,978** also have exactly **0.0** for their total data recharges.

Meaning:

* A second tier of anomalous records with the exact same characteristics as the first group
* Perfect correlation with zero data recharges again indicates a systematic issue
* This is clearly another batch of problematic data points, not legitimate customer behavior

This consistent pattern reinforces that these are data quality issues rather than actual high-value customers. These records should also be adjusted before conducting any revenue analysis to prevent distortion of results and ensure accurate customer segmentation.

The presence of multiple tiers of identical anomalous values suggests a systematic data processing issue that should be investigated with the data provider.

In [92]:
# Replace the values with 0
df['arpu_4g']=df['arpu_4g'].replace(87978,0)

In [93]:
# Checking the quantiles again
df['arpu_4g'].quantile([0.75,0.8,0.9,0.95,0.97,0.99,0.999])

0.750     107.760000
0.800     432.246000
0.900    1803.560000
0.950    2424.072000
0.970    2735.554400
0.990    8705.097343
0.999    8839.721689
Name: arpu_4g, dtype: float64

In [95]:
# Check the churn value for this ARPU
df[df['arpu_4g']>8000]['churn_value'].value_counts()

churn_value
0    16157
1      980
Name: count, dtype: int64

**ARPU 4G Distribution After Second Outlier Treatment**

After removing both tiers of extreme outliers `254,687` and `87,978`, the ARPU 4G distribution has normalized further:

* The `75th` percentile is now `107.76` units
* The `90th` percentile sits at `1,803.56` units
* The highest values `99.9th` percentile have decreased to `8,839.72` units

These changes show:

1. The distribution has become more realistic and analytically useful
2. Each tier of outlier removal further stabilized the percentile values
3. The gap between the `97th` percentile `2,735.55` and the 99th percentile `8,705.10` still suggests a natural breakpoint for high-value customer segmentation
4. The high-end values now form a more gradual progression rather than discrete jumps

The data is now much more suitable for analysis. The remaining high values around `8,800` units appear to be genuine customer behavior rather than data errors, as they form part of a more continuous distribution curve rather than isolated spikes.

It would be valuable to examine the churn status of these remaining high-ARPU customers to understand if high spenders show different retention patterns than average customers.

In [96]:
# Check the value counts of total recharge data at outlying values
df[df['arpu_5g']==254687]['total_rech_data'].value_counts()

total_rech_data
0.0    12614
Name: count, dtype: int64

In [97]:
# Check the value counts of total recharge data at outlying values
df[df['arpu_5g']==87978]['total_rech_data'].value_counts()

total_rech_data
0.0    5130
Name: count, dtype: int64

In [98]:
# Replacing the values with 0 where total recharge data is 0
df['arpu_5g']=df['arpu_5g'].replace([87978,254687],0)

In [99]:
# Check the quantiles of ARPU 5G
df['arpu_5g'].quantile([0.75,0.8,0.9,0.95,0.97,0.99,0.999])

0.750      96.490000
0.800     417.102000
0.900    1797.618000
0.950    2543.904000
0.970    2792.060000
0.990    8587.153966
0.999    8724.440600
Name: arpu_5g, dtype: float64

**ARPU 5G Outlier Treatment**

After examining the ARPU 5G data, I identified the same pattern of outliers as in ARPU 4G:

* 12,614 records with the extreme value of 254,687 units
* All of these records showed zero total data recharges
* Similar anomalies appeared at the 87,978 value

After replacing both problematic values with zero, the ARPU 5G distribution shows a much more realistic pattern:

* The 75th percentile is now 96.49 units
* The 90th percentile sits at 1,797.62 units
* The maximum values (99.9th percentile) have decreased to 8,724.44 units

This cleansed distribution reveals:

1. The ARPU 5G and ARPU 4G distributions now show similar patterns, suggesting internal consistency
2. The outlier removal has created a more reliable foundation for analysis
3. The distribution remains right-skewed but follows a more natural progression
4. The high-end values (around 8,700 units) now appear to be legitimate high-value customers rather than data errors

These adjustments significantly improve data quality for revenue analysis, customer segmentation, and churn prediction models by removing artificial values that would have distorted results.

**Assumptions for CLTV Calculation:**

1. **Incoming Communications:** All incoming calls, messages, and data transfers are considered non-revenue generating, aligning with common telecom practices where charges apply primarily to outbound services.

2. **Customer Support Interactions:** Calls to service centers are excluded from revenue calculations, as these are typically offered free of charge and do not contribute to revenue.

3. **Core Revenue Streams:** The CLTV model focuses on actual revenue-generating activities, including outgoing calls, data usage, subscription fees, and value-added services.

These assumptions ensure that the CLTV calculation accurately reflects genuine revenue sources, providing a clear view of customer value.

In [101]:
# Assuming incoming is free
# We will just keep outgoing and data usage, and premium services columns and assume outgoing to call center is free

# Combining all outgoing call types excluding call center contacts
df['outgoing_calls'] = df['roam_og'] + df['loc_og_t2t'] + df['loc_og_t2m'] + df['loc_og_t2f'] + \
                            df['std_og_t2t'] + df['std_og_t2m'] + df['std_og_t2f'] + \
                            df['isd_og'] + df['spl_og'] + df['og_others']
                            
# Unifying all data consumption into single metric
df['data_usage'] = df['vol_4g'] + df['vol_5g'] + df['streaming_data_consumption']

# Calculate comprehensive revenue metric
df['total_arpu'] = df[['arpu', 'arpu_4g', 'arpu_5g']].sum(axis=1)

In [102]:
# Removing after 0.90 , replacing them with group medians
df['outgoing_calls'].quantile([0.75,0.8,0.9,0.95,0.97,0.99,0.999])

0.750     4436.480000
0.800     5594.251281
0.900     8960.748141
0.950    12312.060683
0.970    14537.184235
0.990    17997.599267
0.999    22152.084652
Name: outgoing_calls, dtype: float64

- The columns `outgoing_calls`has a highly skewed distribution.
- This distribution pattern suggests natural breakpoints for customer segmentation based on calling behavior.
- The substantial gap between the 90th and 99th percentiles indicates a distinct group of high-volume callers.
- For model stability, values above the 90th percentile may require treatment to prevent these extreme users from disproportionately influencing our predictions.

In [103]:
# Removing after 0.90, replacing them with group medians
df['data_usage'].quantile([0.75,0.8,0.9,0.95,0.97,0.99,0.999])

0.750     1125.82000
0.800     2953.23600
0.900    10039.23000
0.950    14873.23200
0.970    17099.29520
0.990    19111.58200
0.999    21748.18856
Name: data_usage, dtype: float64

In [104]:
#  Removing after 0.90, replacing them with group medians
df['total_arpu'].quantile([0.75,0.8,0.9,0.95,0.97,0.99,0.999])

0.750     2181.600000
0.800     3066.298000
0.900     5783.486000
0.950     9176.836704
0.970    10687.852517
0.990    17325.527488
0.999    19254.865964
Name: total_arpu, dtype: float64

- Similarly, the total ARPU distribution reveals that 75% of customers generate revenue below 2,182 units, while the top 10% generate over 5,783 units. 
- The substantial gap between the 90th and 99th percentiles (from 5,783 to 17,326 units) identifies an ultra-high-value customer segment.

- I'll consider capping values above the 90th percentile to the group median for both metrics. This approach preserves the relative ranking of high-value customers while preventing extreme values from disproportionately influencing my predictive models.

In [105]:
# define the columns to check for outliers
cols_to_check = ['outgoing_calls', 'data_usage', 'total_arpu']

# calculate the upper bound for outlier detection
q = 0.90
upper_bound = df[cols_to_check].quantile(q)

# replace values above the upper bound with NaN
df[cols_to_check] = df[cols_to_check].mask(df[cols_to_check] > upper_bound, np.nan)

In [106]:
# Verifying
df['data_usage'].quantile([0.75,0.8,0.9,0.95,0.97,0.99,0.999])

0.750     908.27000
0.800    1026.61000
0.900    3552.40800
0.950    6428.95800
0.970    8309.97120
0.990    9530.69160
0.999    9984.09232
Name: data_usage, dtype: float64

In [ ]:
# # Fill na
# df['data_usage'] = df['data_usage'].fillna(df['data_usage'].median())
# df['outgoing_calls'] = df['outgoing_calls'].fillna(df['outgoing_calls'].median())
# df['total_arpu'] = df['total_arpu'].fillna(df['total_arpu'].median())

### Data type conversion
- Convert data types as needed
    - Convert strings to appropriate categories
    - Ensure numeric fields are properly typed
    - Format dates for analysis

In [116]:
df.columns

Index(['customer_id', 'month', 'month_of_joining', 'zip_code', 'gender', 'age',
       'married', 'dependents', 'number_of_dependents', 'location_id',
       'service_id', 'state', 'county', 'timezone', 'area_codes', 'country',
       'latitude', 'longitude', 'arpu', 'roam_ic', 'roam_og', 'loc_og_t2t',
       'loc_og_t2m', 'loc_og_t2f', 'loc_og_t2c', 'std_og_t2t', 'std_og_t2m',
       'std_og_t2f', 'std_og_t2c', 'isd_og', 'spl_og', 'og_others',
       'loc_ic_t2t', 'loc_ic_t2m', 'loc_ic_t2f', 'std_ic_t2t', 'std_ic_t2m',
       'std_ic_t2f', 'std_ic_t2o', 'spl_ic', 'isd_ic', 'ic_others',
       'total_rech_amt', 'total_rech_data', 'vol_4g', 'vol_5g', 'arpu_5g',
       'arpu_4g', 'night_pck_user', 'fb_user', 'aug_vbc_5g', 'churn_value',
       'referred_a_friend', 'number_of_referrals', 'phone_service',
       'multiple_lines', 'internet_service', 'internet_type',
       'streaming_data_consumption', 'online_security', 'online_backup',
       'device_protection_plan', 'premium_tech_suppo

In [ ]:
useful_columns = [
    # Demographic & Time-based
    'month', 'month_of_joining', 'age', 'gender', 'married', 'dependents',
    # Usage & Revenue Metrics
    'arpu', 'total_rech_amt', 'total_rech_data', 'vol_4g', 'vol_5g',
    'Streaming Data Consumption', 'Satisfaction Score', 'Number of Referrals',
    # Service Features
    'Phone Service', 'internet_service', 'Internet Type', 'Online Security', 
    'Online Backup', 'Device Protection Plan', 'Premium Tech Support',
    'Streaming TV', 'Streaming Movies', 'Streaming Music', 'Unlimited Data',
    'Payment Method', 'offer',
    # Additional revenue features
    'arpu_5g', 'arpu_4g',
    # Target variable
    'Churn Value'
]

In [ ]:
# # Define the variables to loop over (premium and other important variables which can have an impact on ARPU as per hypotheses)

# services = ['Internet Service', 'Online Security', 'Online Backup', 'Device Protection Plan', 'Premium Tech Support',
#             'Streaming TV', 'Streaming Movies', 'Streaming Music', 'Unlimited Data']
# # Encode data
# mapping_dict = {'Yes': 1, 'No': 0}
# for column in services:
#     # replace 'Yes' and 'No' with 1 and 0
#     df[column] = df[column].replace(mapping_dict)

# # checking data types
# df[services].info()

In [ ]:
def convert_data_types(df):
    """
    Convert columns to their appropriate data types, handling NaNs safely.
    """
    # Convert month columns to integers
    df['month'] = pd.to_numeric(df['month'], errors='coerce').astype('Int64')
    df['month_of_joining'] = pd.to_numeric(df['month_of_joining'], errors='coerce').astype('Int64')
    
    # Convert zip_code to int
    df['zip_code'] = df['zip_code'].astype(int)
    
    # Convert age and number_of_dependents to integers
    df['age'] = pd.to_numeric(df['age'], errors='coerce').astype('Int64')
    df['number_of_dependents'] = df['number_of_dependents'].fillna(0).astype(int)
    
    # Convert coordinates to float
    df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
    df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
    
    # Convert usage metrics to float
    usage_columns = [
        'arpu', 'roam_ic', 'roam_og', 'loc_og_t2t', 'loc_og_t2m', 'loc_og_t2f', 
        'loc_og_t2c', 'std_og_t2t', 'std_og_t2m', 'std_og_t2f', 'std_og_t2c', 
        'isd_og', 'spl_og', 'og_others', 'loc_ic_t2t', 'loc_ic_t2m', 'loc_ic_t2f', 
        'std_ic_t2t', 'std_ic_t2m', 'std_ic_t2f', 'std_ic_t2o', 'spl_ic', 
        'isd_ic', 'ic_others', 'total_rech_amt', 'total_rech_data', 'vol_4g', 
        'vol_5g', 'streaming_data_consumption'
    ]
    df[usage_columns] = df[usage_columns].apply(pd.to_numeric, errors='coerce')
    
    # Convert boolean/flag columns to integers
    flag_columns = ['night_pck_user', 'fb_user', 'aug_vbc_5g', 'churn_value']
    for col in flag_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
    
    # Convert referrals and satisfaction score to int
    df['number_of_referrals'] = pd.to_numeric(df['number_of_referrals'], errors='coerce').fillna(0).astype(int)
    df['satisfaction_score'] = pd.to_numeric(df['satisfaction_score'], errors='coerce').astype('Int64')

    return df
convert_data_types(df)

,customer_id,month,month_of_joining,zip_code,gender,age,married,dependents,number_of_dependents,location_id,service_id,state,county,timezone,area_codes,country,latitude,longitude,arpu,roam_ic,roam_og,loc_og_t2t,loc_og_t2m,loc_og_t2f,loc_og_t2c,std_og_t2t,std_og_t2m,std_og_t2f,std_og_t2c,isd_og,spl_og,og_others,loc_ic_t2t,loc_ic_t2m,loc_ic_t2f,std_ic_t2t,std_ic_t2m,std_ic_t2f,std_ic_t2o,spl_ic,isd_ic,ic_others,total_rech_amt,total_rech_data,vol_4g,vol_5g,arpu_5g,arpu_4g,night_pck_user,fb_user,aug_vbc_5g,churn_value,referred_a_friend,number_of_referrals,phone_service,multiple_lines,internet_service,internet_type,streaming_data_consumption,online_security,online_backup,device_protection_plan,premium_tech_support,streaming_tv,streaming_movies,streaming_music,unlimited_data,payment_method,status_id,satisfaction_score,churn_category,churn_reason,customer_status,total_recharge,offer,outgoing_calls,data_usage,total_arpu
0,hthjctifkiudi0,1,1,71638,Female,36,No,No,0,jeavwsrtakgq0,bfbrnsqreveeuafgps0,AR,Chicot County,America/Chicago,870.0,US,33.52,-91.43,273.07,18.88,78.59,280.32,30.970000,5.71,1.790000,25.71,175.56,0.470000,0.0,5.11,0.65,13.99,121.51,168.40,67.61,115.69,52.22,18.71,0.0,0.26,11.53,46.42,18.0,0.0,38.30,219.25,0.00,0.00,0,0,214,1,Yes,9,Yes,Yes,Yes,DSL,27,No,No,Yes,Yes,No,Yes,Yes,Yes,Credit Card,vvhwtmkbxtvsppd52013,3,Competitor,Competitor offered higher download speeds,Churned,18.0,A,617.080000,284.55,273.07
1,uqdtniwvxqzeu1,6,6,72566,Male,36,No,No,0,qcvetdmalnkw1,tkqnsqflrdatnqapsh1,AR,Izard County,America/Chicago,870.0,US,36.22,-92.08,-329.96,69.46,72.08,255.73,148.800000,30.00,7.610000,308.29,265.20,10.820000,0.0,1.23,905.51,1.69,212.93,155.19,29.04,9.15,38.89,0.84,0.0,0.05,32.51,25.53,1183.0,0.0,0.00,0.00,0.00,0.00,0,1,0,0,No,0,Yes,Yes,No,Not Applicable,14,No,Yes,No,No,Yes,No,No,No,Bank Withdrawal,jucxaluihiluj82863,4,Not Applicable,Not Applicable,Stayed,1183.0,F,1999.350000,14.00,-329.96
2,uqdtniwvxqzeu1,7,6,72566,Male,36,No,No,0,qcvetdmalnkw1,tkqnsqflrdatnqapsh1,AR,Izard County,America/Chicago,870.0,US,36.22,-92.08,101.22,1012.60,115.26,52.95,1151.693063,103.28,15.710000,244.20,15.19,61.837211,0.0,13.14,455.15,115.63,121.80,699.39,44.49,83.59,914.70,13.25,0.0,0.06,13.05,5.62,295.0,7.0,14.83,967.95,-9.40,106.30,1,1,85,0,Yes,6,Yes,No,Yes,Cable,82,No,No,Yes,No,Yes,No,No,Yes,Credit Card,vjskkxphumfai57182,3,Not Applicable,Not Applicable,Stayed,302.0,No Offer,2328.330274,1064.78,198.12
3,uqdtniwvxqzeu1,8,6,72566,Male,36,No,No,0,qcvetdmalnkw1,tkqnsqflrdatnqapsh1,AR,Izard County,America/Chicago,870.0,US,36.22,-92.08,215.48,84.18,99.85,140.51,4006.990000,280.86,6.330000,346.14,103.15,183.530000,0.0,33.88,495.60,14.01,658.96,195.02,144.11,50.18,2.35,623.94,0.0,0.07,69.13,10.62,354.0,1.0,264.90,268.11,-5.15,77.53,0,1,268,0,Yes,10,Yes,No,Yes,Fiber Optic,57,No,No,Yes,No,Yes,No,No,Yes,Wallet Balance,cdwbcrvylqca53109,4,Not Applicable,Not Applicable,Stayed,355.0,J,5704.520000,590.01,287.86
4,uqdtniwvxqzeu1,9,6,72566,Male,36,No,No,0,qcvetdmalnkw1,tkqnsqflrdatnqapsh1,AR,Izard County,America/Chicago,870.0,US,36.22,-92.08,636.55,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.0,0.00,0.00,0.00,138.85,201.92,19.89,15.91,23.78,16.01,0.0,0.03,64.35,36.18,0.0,0.0,52.78,370.59,0.00,0.00,0,0,399,0,Yes,1,No,No,Yes,Fiber Optic,38,No,No,No,No,No,Yes,No,Yes,Credit Card,whqrmeulitfj98550,1,Not Applicable,Not Applicable,Stayed,0.0,No Offer,0.000000,461.37,636.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653748,ksdlqrgoihlmp162977,8,6,98932,Female,22,Yes,No,0,nxoadzllyxzg162977,bxpevtcjeectllyrdt162977,WA,Yakima County,America/Los_Angeles,509.0,US,46.32,-120.15,195.90,45.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.0,0.00,0.00,0.00,3990.18,2121.43,969.35,19

## Exploratory Data Analysis

## Feature Engineering

## Data Preprocessing and Transformation

- Normalize/standardize numeric features
    - Apply scaling (min-max, standard scaling)
    - Apply log transformations for skewed distributions

- Encode categorical variables
    - One-hot encoding for nominal categories
    - Label encoding for ordinal categories
    - Target encoding for high-cardinality features


In [ ]:
# numerical_features_list = ["tenure", "MonthlyCharges", "TotalCharges"]

# for col in df.columns:
#   if col not in numerical_features_list:
#     print(col, df[col].unique())
#     print("-"*50)

In [122]:
# Step 1: Define our column groups based on previous organization
customer_demographics = ['customer_id', 'gender', 'age', 'married', 'dependents', 
                         'number_of_dependents', 'referred_a_friend', 'number_of_referrals', 
                         'satisfaction_score']

location_info = ['zip_code', 'location_id', 'state', 'county', 'timezone', 
                'area_codes', 'country', 'latitude', 'longitude']

account_info = ['month', 'month_of_joining', 'service_id', 'customer_status', 
               'status_id', 'offer']

service_subscriptions = ['phone_service', 'multiple_lines', 'internet_service', 
                        'internet_type', 'online_security', 'online_backup', 
                        'device_protection_plan', 'premium_tech_support', 
                        'streaming_tV', 'streaming_movies', 'streaming_music', 
                        'unlimited_data', 'night_pck_user', 'fb_user']

voice_usage = ['outgoing_calls', 'loc_og_t2t', 'loc_og_t2m', 'loc_og_t2f', 
              'loc_og_t2c', 'std_og_t2t', 'std_og_t2m', 'std_og_t2f', 
              'std_og_t2c', 'isd_og', 'spl_og', 'og_others', 'loc_ic_t2t', 
              'loc_ic_t2m', 'loc_ic_t2f', 'std_ic_t2t', 'std_ic_t2m', 
              'std_ic_t2f', 'std_ic_t2o', 'spl_ic', 'isd_ic', 'ic_others', 
              'roam_ic', 'roam_og']

data_usage = ['data_usage', 'vol_4g', 'vol_5g', 'streaming_data_consumption', 
             'total_rech_data', 'aug_vbc_5g']

revenue_metrics = ['arpu', 'arpu_4g', 'arpu_5g', 'total_arpu', 'total_rech_amt', 
                  'payment_method']

churn_info = ['churn_value', 'churn_category', 'churn_reason']

In [124]:
df['churn_value'].value_counts()

churn_value
0    623888
1     29865
Name: count, dtype: int64

In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 653753 entries, 0 to 653752
Data columns (total 78 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   customer_id                 653753 non-null  object 
 1   month                       653753 non-null  Int64  
 2   month_of_joining            653753 non-null  Int64  
 3   zip_code                    653753 non-null  object 
 4   gender                      653753 non-null  object 
 5   age                         653753 non-null  Int64  
 6   married                     653753 non-null  object 
 7   dependents                  653753 non-null  object 
 8   number_of_dependents        653753 non-null  int64  
 9   location_id                 653753 non-null  object 
 10  service_id                  653753 non-null  object 
 11  state                       653753 non-null  object 
 12  county                      653753 non-null  object 
 13  timezone      

In [ ]:
# # Introduce some df quality issues for demonstration
# # Missing values
# for col in ['Age', 'Number of Dependents', 'arpu', 'df_usage']:
#     missing_indices = np.random.choice(df, size=int(df * 0.05), replace=False)
#     df[col] = pd.Series(df[col])
#     df[col].iloc[missing_indices] = np.nan